In [1]:
!curl -s -X DELETE http://localhost:9200/ubi_queries/
!curl -s -X DELETE http://localhost:9200/ubi_events/

{"acknowledged":true}{"acknowledged":true}

In [2]:
from time import time
import json
import random
import uuid

from opensearchpy import OpenSearch

client = OpenSearch("http://localhost:9200", use_ssl=False)

## Parameterize the generation of UBI queries and events

In [3]:
search_terms = ['computer', 'laptop', 'notebook', 'desk', 'power plug']
actions = ["click_through", "add_to_cart", "click", "watch", "view", "purchase"]

object_id_field = "ean"
item_description_field = "title"
index_name = "ecommerce"
number_of_user_searches = 1000
page_size = 20
max_number_events_for_each_search = 10

## Generate the UBI queries and events

In [4]:
for x in range(number_of_user_searches):

    random_search_term_index = random.randint(0, len(search_terms) - 1)
    random_search_term = search_terms[random_search_term_index]
    
    client_id = str(uuid.uuid4())
    query_id = str(uuid.uuid4())
    
    query = {
       "from": 0,
        "size": page_size,
       "query": {
         "match_all": {}
       },
        "ext": {
            "ubi": {
                "client_id": client_id,
                "query_id": query_id,
                "user_query": random_search_term
            }
        }
     }
    
    query_response = client.search(
        body = query,
        index = index_name
    )

    random_number_of_events = random.randint(0, max_number_events_for_each_search - 1)

    for y in range(random_number_of_events):
            
        random_search_result_index = random.randint(0, page_size - 1)
        random_action_index = random.randint(0, len(actions) - 1)
        session_id = str(uuid.uuid4())
        
        #print("random_action_index = " + str(random_action_index))
        #print("random_search_result_index = " + str(random_search_result_index))

        ubi_event = {
            "action_name": actions[random_action_index],
            "client_id": client_id,
            "query_id": query_id,
            "message_type": None,
            "message": None,
            "timestamp": time(),
            "event_attributes": {
                "object": {
                    "object_id_field": "ean",
                    "object_id": query_response["hits"]["hits"][random_search_result_index]["_source"][object_id_field],
                    "description": query_response["hits"]["hits"][random_search_result_index]["_source"][item_description_field]
                },
                "position": {
                    "index": random_search_result_index
                },
                "session_id": session_id
            }
        }
    
        event_id = str(uuid.uuid4())
        
        response = client.index(
            body = ubi_event,
            index = "ubi_events",
            id = event_id,
            refresh = True
        )

In [5]:
!curl -s http://localhost:9200/ubi_queries/_count | jq
!curl -s http://localhost:9200/ubi_events/_count | jq

{
  "count": 1000,
  "_shards": {
    "total": 1,
    "successful": 1,
    "skipped": 0,
    "failed": 0
  }
}
{
  "count": 4415,
  "_shards": {
    "total": 1,
    "successful": 1,
    "skipped": 0,
    "failed": 0
  }
}


In [6]:
!curl -s http://localhost:9200/ubi_queries/_search | jq

{
  "took": 0,
  "timed_out": false,
  "_shards": {
    "total": 1,
    "successful": 1,
    "skipped": 0,
    "failed": 0
  },
  "hits": {
    "total": {
      "value": 1000,
      "relation": "eq"
    },
    "max_score": 1,
    "hits": [
      {
        "_index": "ubi_queries",
        "_id": "pBAEw5EB7SVFyyWb2-0L",
        "_score": 1,
        "_source": {
          "timestamp": 1725553757874,
          "queryId": "9714840d-9f32-4f74-9b0c-7f5821d20c0a",
          "clientId": "f69d0c7f-0b59-4df5-b825-cd482fa836fd",
          "userQuery": "computer",
          "query": "{\"from\":0,\"size\":20,\"query\":{\"match_all\":{\"boost\":1.0}},\"ext\":{\"query_id\":\"9714840d-9f32-4f74-9b0c-7f5821d20c0a\",\"user_query\":\"computer\",\"client_id\":\"f69d0c7f-0b59-4df5-b825-cd482fa836fd\",\"object_id_field\":null,\"query_attributes\":{}}}",
          "queryAttributes": {},
          "queryResponse": {
            "queryId": "9714840d-9f32-4f74-9b0c-7f5821d20c0a",
            "queryResponseId": "

In [7]:
!curl -s http://localhost:9200/ubi_events/_search | jq

{
  "took": 0,
  "timed_out": false,
  "_shards": {
    "total": 1,
    "successful": 1,
    "skipped": 0,
    "failed": 0
  },
  "hits": {
    "total": {
      "value": 4415,
      "relation": "eq"
    },
    "max_score": 1,
    "hits": [
      {
        "_index": "ubi_events",
        "_id": "13e874a7-5a55-45dd-8f66-aa863e02d59c",
        "_score": 1,
        "_source": {
          "action_name": "purchase",
          "client_id": "185272b0-c14f-4f1e-9247-1e69fcc70105",
          "query_id": "923ca6ea-0e0a-4e9a-b0ad-d53021cb321a",
          "message_type": null,
          "message": null,
          "timestamp": 1725553758.460172,
          "event_attributes": {
            "object": {
              "object_id_field": "ean",
              "object_id": [
                "0731304258193"
              ],
              "description": "APC IT Power Distribution Module 3 Pole 5 Wire 32A IEC309 620cm power distribution unit (PDU)"
            },
            "position": {
              "index